In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

In [5]:
import pandas as pd
df=pd.read_csv(r"C:\Users\jeeva\Desktop\Wind Turbine\Model Building\Wind_Turbine_failure_After_Preprocessing.csv")
df

,Wind_speed,Power,Nacelle_ambient_temperature,Generator_bearing_temperature,Gear_oil_temperature,Ambient_temperature,Rotor_Speed,Nacelle_temperature,Bearing_temperature),Generator_speed,Yaw_angle,Wind_direction,Wheel_hub_temperature,Gear_box_inlet_temperature,Failure_status
0,0.262925,0.389423,0.666401,0.520292,0.188158,0.538156,0.298285,0.633544,0.625000,0.520057,0.080149,0.364537,0.489121,0.242608,No_failure
1,0.982171,0.011220,0.000000,0.225072,0.643013,0.024046,0.053718,0.000000,0.116923,0.005979,0.545414,0.794844,0.066476,0.578307,Failure
2,0.389520,0.333851,0.733446,0.460442,0.000000,0.488129,0.298966,0.472697,0.574760,0.495130,0.082661,0.983137,0.728228,0.121434,No_failure
3,0.936187,0.000000,0.000000,0.081055,0.777958,0.172658,0.052741,0.039929,0.000000,0.080247,0.922165,1.000000,0.074493,0.383299,Failure
4,0.486451,0.551490,0.481618,0.553906,0.431672,0.601514,0.247183,0.532911,0.375000,0.384667,0.075022,0.441862,0.463140,0.041690,No_failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,0.106153,1.000000,0.856370,1.000000,0.763001,1.000000,0.798245,1.000000,1.000000,0.982118,0.833781,1.000000,0.956242,1.000000,Failure
3596,0.170081,0.459211,0.402909,0.383469,0.099461,0.427102,0.116053,0.518656,0.375000,0.443175,0.054447,0.450900,0.458285,0.162595,No_failure
3597,0.165581,0.563065,0.427159,0.484266,0.145010,0.589248,0.270338,0.329149,0.462422,0.404546,0.007179,0.420471,0.633114,0.000000,No_failure
3598,0.502258,0.461578,0.389078,0.417591,0.300886,0.268962,0.477017,0.505087,0.625000,0.570593,0.014102,0.381109,0.610918,0.083783,No_failure


In [6]:
#df = df.drop(columns=['date'])

In [7]:
# Split the data into features and target
X = df.drop(columns=['Failure_status'])  # Features
y = df['Failure_status']  # Target variable

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## MultinomialNB Base model 

In [9]:
# Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
mnb_score = mnb.score(X_test, y_test)
print("Multinomial Naive Bayes Accuracy:", mnb_score)

Multinomial Naive Bayes Accuracy: 0.9416666666666667


## Hyperopt

In [15]:
from sklearn.naive_bayes import MultinomialNB
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
from sklearn.metrics import accuracy_score
import numpy as np

# Define the parameter space
param_space = {
    'alpha': hp.loguniform('alpha', np.log(0.01), np.log(1.0)),
    'fit_prior': hp.choice('fit_prior', [True, False])
}

# Define the objective function
def objective(params):
    model = MultinomialNB(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return {'loss': -accuracy, 'params': params, 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best = fmin(fn=objective,
            space=param_space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

# Get the best hyperparameters
best_params = space_eval(param_space, best)

# Initialize the MultinomialNB model with the best hyperparameters
model = MultinomialNB(**best_params)
print(model)

# Fit the model
model.fit(X_train, y_train)

#Predict on Train Data
y_pred_train = model.predict(X_train)
Train_accuracy = accuracy_score(y_train, y_train)
print("Multinomial Naive Bayes Train_accuracy:", Train_accuracy)
# Evaluate on test data
y_pred = model.predict(X_test)


Test_accuracy = accuracy_score(y_test, y_pred)
print("Multinomial Naive Bayes Test_accuracy:", Test_accuracy)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 40.52trial/s, best loss: -0.9986111111111111]
MultinomialNB(alpha=0.3440848163628876, fit_prior=False)
Multinomial Naive Bayes Train_accuracy: 1.0
Multinomial Naive Bayes Test_accuracy: 0.9986111111111111


## GridSearchCV

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid = {
    'alpha': [0.01, 0.1, 1.0],
    'fit_prior': [True, False]
}

# Create the MultinomialNB model
model = MultinomialNB()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Initialize the MultinomialNB model with the best hyperparameters
model = MultinomialNB(**best_params)

# Fit the model
model.fit(X_train, y_train)

# Evaluate on train data
y_pred_train = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Multinomial Naive Bayes Train Accuracy:", train_accuracy)

# Evaluate on test data
y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Multinomial Naive Bayes Test Accuracy:", test_accuracy)

Best Hyperparameters: {'alpha': 0.01, 'fit_prior': False}
Multinomial Naive Bayes Train Accuracy: 0.9972222222222222
Multinomial Naive Bayes Test Accuracy: 0.9986111111111111
